In [2]:
def process_real_estate_data(service_key, gu_code, n=10000, months=12, filter_region='대치동'):
    """
    Process real estate data for sales and rent for the specified region.

    Args:
        service_key (str): Service key for API requests.
        gu_code (int): Region code for data retrieval.
        n (int): Number of records to fetch per request. Default is 10,000.
        months (int): Number of past months to fetch data for. Default is 3.
        filter_region (str): Region name to filter (e.g., '대치동'). Default is '대치동'.

    Returns:
        pd.DataFrame: Processed DataFrame combining sales and rent data.
    """
    from 매매가 import fetch_apt_sale_data
    from 전월세 import fetch_apt_rent_data
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    import pandas as pd

    # Calculate date ranges for the last 'months'
    end_date = datetime.today()
    dates = [(end_date - relativedelta(months=i)).strftime('%Y%m') for i in range(months)]

    # Initialize empty DataFrames
    df_sale_combined = pd.DataFrame()
    df_rent_combined = pd.DataFrame()

    # Fetch and concatenate sales and rental data for the given months
    for date in dates:
        df_sale = fetch_apt_sale_data(service_key, gu_code, date, n)
        df_rent = fetch_apt_rent_data(service_key, gu_code, date, n)

        df_sale_combined = pd.concat([df_sale_combined, df_sale], ignore_index=True)
        df_rent_combined = pd.concat([df_rent_combined, df_rent], ignore_index=True)

    # Filter data for the specified region
    df_sale_deachi = df_sale_combined[df_sale_combined['dong_name'].str.contains(filter_region)]
    df_rent_deachi = df_rent_combined[df_rent_combined['dong_name'].str.contains(filter_region)]

    # Create 'deal_date' column
    df_sale_deachi['deal_date'] = (
        df_sale_deachi['deal_year'].astype(str) +
        df_sale_deachi['deal_month'].astype(str).str.zfill(2) +
        df_sale_deachi['deal_day'].astype(str).str.zfill(2)
    )

    df_rent_deachi['deal_date'] = (
        df_rent_deachi['deal_year'].astype(str) +
        df_rent_deachi['deal_month'].astype(str).str.zfill(2) +
        df_rent_deachi['deal_day'].astype(str).str.zfill(2)
    )

    # Create combined apartment name and area column
    df_sale_deachi['apt_name_area'] = (
        df_sale_deachi['apt_name'] + '_' + df_sale_deachi['apt_userArea'].astype(str)
    )

    df_rent_deachi['apt_name_area'] = (
        df_rent_deachi['apt_name'] + '_' + df_rent_deachi['apt_userArea'].astype(str)
    )

    # Extract relevant columns
    df_sale_deachi = df_sale_deachi[
        ['apt_name_area', 'apt_floor', 'deal_amount', 'deal_date']
    ]
    df_rent_deachi = df_rent_deachi[
        ['apt_name_area', 'apt_floor', 'deal_deposit', 'deal_rent', 'deal_date']
    ]

    # Remove duplicates and sort by apartment name
    df_sale_deachi = df_sale_deachi.sort_values('deal_date', ascending=False).drop_duplicates(subset='apt_name_area')
    df_rent_deachi = df_rent_deachi.sort_values('deal_date', ascending=False).drop_duplicates(subset='apt_name_area')

    # Combine sales and rental data into one DataFrame
    df_combined_deachi = pd.concat([df_sale_deachi, df_rent_deachi], ignore_index=True)

    # Adjust column order and sort by apartment name
    df_combined_deachi = df_combined_deachi[
        ['apt_name_area', 'apt_floor', 'deal_amount', 'deal_deposit', 'deal_rent', 'deal_date']
    ]
    df_combined_deachi = df_combined_deachi.sort_values('apt_name_area', ascending=True)

    return df_combined_deachi

service_key = "sgOQlwpackhO01oaCAAcwfJy4IGwN%2F2uLcXifUcmx%2FIwLRwPkaBTgz9kBtg5oQg8YRxPZICHxtKF%2BsXKn4rf%2Fw%3D%3D"
gu_code = 11680
n = 10000
months = 12
filter_region = '대치동'

df_combined_result = process_real_estate_data(
    service_key=service_key,
    gu_code=gu_code,
    n=n,
    months=months,
    filter_region=filter_region
)

df_combined_result

C:\Users\yoonh\PycharmProjects\PythonProject4\Portfolio\전월세.py:33: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  house = BeautifulSoup(result, 'lxml')
[NbConvertApp] Converting notebook 전월세.ipynb to script
[NbConvertApp] Writing 2909 bytes to 전월세.py
C:\Users\yoonh\PycharmProjects\PythonProject4\Portfolio\매매가.py:35: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure

,apt_name_area,apt_floor,deal_amount,deal_deposit,deal_rent,deal_date
4,개포우성1_127.61,4,420000.0,NaN,NaN,20250112
146,개포우성1_127.61,8,NaN,100000.0,0.0,20250117
285,개포우성1_158.54,5,NaN,166000.0,0.0,20240520
72,개포우성1_189.98,7,560000.0,NaN,NaN,20240725
282,개포우성1_189.98,7,NaN,140000.0,340.0,20240522
...,...,...,...,...,...,...
314,현대썬앤빌테헤란_45.89,7,NaN,52000.0,0.0,20240311
309,현대썬앤빌테헤란_48.27,7,NaN,55000.0,0.0,20240321
263,화인하이빌_117.98,1,NaN,55200.0,0.0,20240718
274,화인하이빌_132.59,6,NaN,80000.0,0.0,20240621


In [2]:
!jupyter nbconvert --to script --output "매매가전월세통합" 매매가전월세통합.ipynb


[NbConvertApp] Converting notebook 매매가전월세통합.ipynb to script
[NbConvertApp] Writing 4075 bytes to 매매가전월세통합.py
